In [77]:
import numpy as np
import pandas as pd

In [78]:
# Set up hyperparameters
np.random.seed(24)
epochs = 1000
learnrate = 0.5

In [79]:
# Data preparation
admissions = pd.read_csv('gradient_descent.csv')

# Make dummy variables for rank
data = pd.concat([admissions, pd.get_dummies(admissions['rank'], prefix='rank')], axis=1)
data = data.drop('rank', axis=1)

# Standarize features
for field in ['gre', 'gpa']:
    mean, std = data[field].mean(), data[field].std()
    data.loc[:,field] = (data[field]-mean)/std

# Split off random 10% of the data for testing
sample = np.random.choice(data.index, size=int(len(data)*0.9), replace=False)
data, test_data = data.ix[sample], data.drop(sample)

# Split into features and targets
features, targets = data.drop('admit', axis=1), data['admit']
features_test, targets_test = test_data.drop('admit', axis=1), test_data['admit']

n_records, n_features = features.shape

In [81]:
print('Features:')
print(features[0:5])
print()
print('Targets:')
print(targets[0:5])

Features:
          gre       gpa  rank_1  rank_2  rank_3  rank_4
115  0.279614  1.576859     0.0     0.0     1.0     0.0
186 -0.239793  0.578348     0.0     0.0     1.0     0.0
366 -1.105469 -1.208461     0.0     0.0     1.0     0.0
81   0.279614 -0.840588     0.0     1.0     0.0     0.0
314 -0.412928  0.184199     0.0     0.0     0.0     1.0

Targets:
115    0
186    0
366    0
81     0
314    0
Name: admit, dtype: int64


In [82]:
def sigmoid(x):
    """
    Calculate sigmoid
    """
    return 1 / (1 + np.exp(-x))

In [83]:
# Initialize weights
weights = np.random.normal(scale=1 / n_features**.5, size=n_features)

last_loss = None

for e in range(epochs):
    del_w = np.zeros(weights.shape)
    for x, y in zip(features.values, targets):
        # Loop through all records, x is the input, y is the target

        # TODO: Calculate the output
        output = sigmoid(np.dot(weights, x))

        # TODO: Calculate the error
        error = y - output

        # TODO: Calculate change in weights
        del_w += error * x * output * (1 - output)

        # TODO: Update weights
    weights += learnrate * del_w / n_records

    # Printing out the mean square error on the training set
    if e % (epochs / 10) == 0:
        out = sigmoid(np.dot(features, weights))
        loss = np.mean((out - targets) ** 2)
        if last_loss and last_loss < loss:
            print("Train loss: ", loss, "  WARNING - Loss Increasing")
        else:
            print("Train loss: ", loss)
        last_loss = loss

Train loss:  0.2335730151102724
Train loss:  0.20852680826853684
Train loss:  0.2020865394494544
Train loss:  0.19983464869528578
Train loss:  0.19888292837168906
Train loss:  0.19842779228840485
Train loss:  0.1981907670213906
Train loss:  0.19805943833197331
Train loss:  0.19798319411827955
Train loss:  0.19793730938162654


In [84]:
# Calculate accuracy on test data
tes_out = sigmoid(np.dot(features_test, weights))
predictions = tes_out > 0.5
accuracy = np.mean(predictions == targets_test)
print("Prediction accuracy: {:.3f}".format(accuracy))

Prediction accuracy: 0.775
